In [1]:
import pandas as pd
import os
import random

In [2]:
df = pd.read_excel('salidas/03-ModeloFinal.xlsx')
df

,localidad,ANO,MES,edad,sexo,pm25_pred,pm25_pred_prom,estatus,casado,PM25_base,...,EstCiv_6,tiempo2,tiempo3,tiempo4,NoViudo,NoSubsidiado,pm25_prom,HR,lcl,ucl
0,6,2019,4,21,0,19.127556,22.442510,1,1,1,...,0,136,12.307692,12.346154,1,1,-22.442510,0.624067,0.609552,0.638581
1,14,2013,2,24,1,17.962831,17.978268,1,1,1,...,0,62,6.153846,6.153846,0,1,-17.978268,1.181113,1.161386,1.200841
2,10,2021,7,25,0,16.616410,17.931989,1,1,1,...,1,163,14.538462,14.576923,1,1,-17.931989,0.911433,0.898613,0.924252
3,6,2018,2,24,1,19.746750,22.743869,1,1,1,...,0,122,11.153846,11.153846,0,1,-22.743869,0.674364,0.656525,0.692203
4,16,2013,1,22,0,19.944206,20.105310,1,1,1,...,0,61,6.076923,6.115385,1,1,-20.105310,1.122134,1.107997,1.136271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106380,18,2015,1,22,0,19.587503,20.888093,1,1,1,...,0,85,8.076923,8.115385,0,0,-20.888093,0.891396,0.888834,0.893957
106381,19,2015,1,24,0,28.290408,28.507551,1,1,1,...,0,85,8.076923,8.115385,1,0,-28.507551,1.070014,1.060740,1.079289
106382,4,2019,11,22,0,12.655908,15.396644,1,1,1,...,1,143,12.846154,12.884615,1,1,-15.396644,0.704829,0.691540,0.718118
106383,8,2021,7,17,0,23.168129,25.355268,1,1,1,...,0,163,14.538462,14.576923,1,1,-25.355268,0.763299,0.748285,0.778314


In [3]:
df000 = df[['pm25_pred','HR']].copy()
df00 = df000[df000['pm25_pred']>=15].copy()
df00 = df000[~((df000['pm25_pred']<=23) &  (df000['HR']>= 1.4))]
df00 = df00[~((df00['pm25_pred']>23) &  (df00['HR']<= 1.0))]
df00.sort_values(by = 'pm25_pred',inplace = True,ignore_index =True)
#df00 = df000.copy()
df00

,pm25_pred,HR
0,11.066298,0.782680
1,11.066298,0.798164
2,11.066298,0.797176
3,11.066298,0.812947
4,11.066298,0.797176
...,...,...
84136,33.776185,1.693942
84137,33.776185,1.661080
84138,33.776185,1.725316
84139,33.776185,1.725316


In [4]:
df_final = pd.DataFrame(columns = df00.columns.to_list())
df_final

,pm25_pred,HR


In [5]:
pm25_min = df00['pm25_pred'][0]
pm25_max = df00['pm25_pred'][len(df00)-1]
n = 5
m = 4
salto = (pm25_max-pm25_min)/n
salto

4.541977505295583

In [6]:
pm25 = pm25_min
for pm25 in range(int(pm25_min),int(pm25_max)):
    ####SELECCION DE DATOS
    df_grupo = df00[(df00['pm25_pred'] >=pm25) & 
                    (df00['pm25_pred'] <pm25+salto)].copy()
    ###ASIGNAR INDICES DE FORMA ALEATORIA 
    s = []
    for x in range(0,len(df_grupo)):
        s.append(random.randint(0,len(df_grupo)))
    df_grupo['orden'] = s
    df_grupo.sort_values('orden',inplace = True,ignore_index=True)
    df_grupo = df_grupo[['pm25_pred','HR']].copy()
    k = int(len(df_grupo)/m)
    pm25 += salto
    
    for i in range(0,len(df_grupo)):
        if i%k == 0:
        #print(i)
        #df1 = df10[i:i+k].groupby('pm25_pred').median().reset_index()
            df0 = df_grupo[i:i+k]
            df1 = pd.DataFrame({df00.columns.to_list()[0] : [df0.pm25_pred.mean()], 
                   df00.columns.to_list()[1] : [df0.HR.median()]})
            df_final = pd.concat([df_final,df1],ignore_index=True)
        #print(df1)
        else:
            continue

In [7]:
print(len(df_final))
df_final = df_final[df_final['HR']<1.8]
df_final = df_final[df_final['HR']>0.8]
df_final.sort_values(by = 'pm25_pred',inplace = True,ignore_index =True)
minpm25 = df_final['HR'][0]
df_final['HR'] = df_final['HR']/minpm25
print(len(df_final))


105
104


In [8]:
#df_final.to_excel('salidas/Pruebas/06-Promedio.xlsx',index=False)
df_final.to_excel('salidas/04-Promedio.xlsx',index=False)